# Configuração

Importa dados sintéticos

In [ ]:
# Instalar pacotes necessários
!pip install transformers datasets peft bitsandbytes accelerate --quiet
!pip install sentencepiece --quiet
!pip install --upgrade transformers

# Imports principais
import requests
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, pipeline
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import csv

# Download do CSV do Drive
file_id = '1aRRsf2lvAawmN4Jmr4Vn0V17Y7FO6z2r'
url = f'https://drive.google.com/uc?export=download&id={file_id}'

response = requests.get(url)
if response.status_code == 200:
    csv_text = response.text
    with open("dataset.csv", "w", encoding="utf-8") as f:
        f.write(csv_text)
    print("Arquivo baixado e salvo como dataset.csv")
else:
    raise Exception(f"Erro ao acessar arquivo: {response.status_code}")

# Carregar o CSV em DataFrame
# Lista para armazenar os dados
dados = []

# Abrir e ler o arquivo CSV manualmente
with open("dataset.csv", encoding="utf-8") as f:
    leitor = csv.reader(f, delimiter=',', quotechar='"')
    for linha in leitor:
        # Ignora linhas mal formatadas (ex: incompletas)
        if len(linha) == 2:
            dados.append(linha)

# Criar o DataFrame
df = pd.DataFrame(dados, columns=["pergunta", "resposta"])

# Verificar os dados
print(df.head())
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig, pipeline
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

# Pré-processamento
def preprocess(row):
    return f"Pergunta: {row['pergunta']} Resposta: {row['resposta']}"

df["text"] = df.apply(preprocess, axis=1)
dataset = Dataset.from_pandas(df[["text"]])

# Tokenizador
model_name = "bigscience/bloom-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# GPT2 não tem token de padding por padrão
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Função de tokenização corrigida
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Quantização com BitsAndBytesConfig (substitui load_in_8bit)
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
)

# Modelo
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)

# Configuração LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Argumentos de treino
training_args = TrainingArguments(
    output_dir="./qlora-br-china",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=1,
    report_to="none",
    push_to_hub=False,
    remove_unused_columns=False,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=lambda data: {
        "input_ids": torch.tensor([f["input_ids"] for f in data]),
        "attention_mask": torch.tensor([f["attention_mask"] for f in data]),
        "labels": torch.tensor([f["input_ids"] for f in data]),  # causal LM usa input_ids como labels
    },
)

# Treinamento
trainer.train()

# Salvar modelo LoRA treinado
model.save_pretrained("./qlora-br-china")

# -------- Inferência --------

# Modelo original
pipe_orig = pipeline("text-generation", model=model_name, tokenizer=tokenizer, device=0)

# Modelo com QLoRA
pipe_qlora = pipeline("text-generation", model="./qlora-br-china", tokenizer=tokenizer, device=0)
perguntas = [
    "Qual é a importância do acordo de exportação de café entre Brasil e China?",
    "Como a parceria China-Brasil pode impactar o setor de tecnologia em 2025?",
    "Quais são os principais investimentos chineses previstos no Brasil para 2025?",
    "Como o comércio bilateral entre Brasil e China deve evoluir até 2025?",
    "Quais setores brasileiros terão maior benefício da cooperação com a China em 2025?",
    "Qual o papel da China na infraestrutura brasileira até 2025?",
    "Como a relação diplomática entre Brasil e China deve se fortalecer em 2025?",
    "Quais desafios podem surgir na relação econômica entre Brasil e China em 2025?",
    "Como o acordo comercial com a China pode afetar a agricultura brasileira em 2025?",
    "Quais oportunidades de inovação surgirão com a cooperação Brasil-China até 2025?"
]

def extrair_resposta_curta(texto):
    parte_resposta = texto.split("Resposta:")[-1].strip()
    primeira_frase = parte_resposta.split('.')[0]
    return primeira_frase + '.'

for pergunta in perguntas:
    entrada = f"Pergunta: {pergunta} Resposta:"

    print(f"\nPergunta: {pergunta}")

    res_orig = pipe_orig(entrada, max_length=50, do_sample=True, top_p=0.9, temperature=0.8)
    resposta_orig = extrair_resposta_curta(res_orig[0]["generated_text"])
    print("Resposta modelo original:", resposta_orig)

    res_qlora = pipe_qlora(entrada, max_length=50, do_sample=True, top_p=0.9, temperature=0.8)
    resposta_qlora = extrair_resposta_curta(res_qlora[0]["generated_text"])
    print("Resposta modelo QLoRA:", resposta_qlora)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 61.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found